In [1]:
import requests
import pyodbc
import pandas as pd
import os
import numpy as np
from scipy.spatial.distance import jaccard

In [3]:
def get_hybrid_recomendation(userId, meditationProgramId, count):
    get_recommendations_url = 'http://localhost:5000/get_recommendations'
    params = {
        'userId': str(userId),
        'meditationProgramId': str(meditationProgramId),
        'count': str(count)
    }
    response = requests.get(get_recommendations_url, params=params)

    if response.status_code == 200:
        recommendations_json = response.json()
        print('Recommendations:', recommendations_json)
    else:
        print('Error:', response.json())
        
    return recommendations_json.get('recommendations', [])

In [4]:
userId = 17
meditationProgramId = 6
count = 10
recommendations = get_hybrid_recomendation(userId, meditationProgramId, count)

Recommendations: {'recommendations': [8, 7, 19, 6, 9, 17, 18, 27, 30, 1]}


In [95]:
conn = pyodbc.connect("DRIVER={SQL Server};SERVER=DESKTOP-7S5VEFS\\SQLEXPRESS; DATABASE=NewMedit;Trusted_Connection=True;")
cursor = conn.cursor()
cursor.execute("SELECT MeditationProgramId FROM Ratings WHERE UserId = ?", (userId))
results = cursor.fetchall()
actual_items = [row[0] for row in results]
cursor.close()
conn.close()
print(actual_items)

[10, 26, 13, 4, 11, 30, 14, 28, 16, 6, 9, 8, 5, 25, 2, 22, 17, 7, 24, 19, 18]


In [96]:
def precision_recall_at_k(recommendations, actual_items, k):
    recommendations = recommendations[:k]
    num_hit = len(set(recommendations) & set(actual_items))
    
    precision = num_hit / k if k != 0 else 0
    recall = num_hit / len(actual_items) if len(actual_items) != 0 else 0
    
    return precision, recall

In [97]:
precision, recall = precision_recall_at_k(recommendations, actual_items, count)
print(f"precision -- {precision}")
print(f"recall -- {recall}")

precision -- 0.8
recall -- 0.38095238095238093


In [98]:
def ndcg_at_k(recommendations, actual_items, k):
    dcg = 0
    idcg = 0
    # Обчислюємо DCG (дисконтований кумулятивний приріст)
    for i, item in enumerate(recommendations[:k]):
        if item in actual_items:
            dcg += 1 / np.log2(i + 2)
    # Обчислюємо IDCG (ідеальний дисконтований кумулятивний приріст)
    for i in range(min(k, len(actual_items))):
        idcg += 1 / np.log2(i + 2)
    # Повертаємо нормалізований DCG
    return dcg / idcg if idcg > 0 else 0

In [99]:
print(f"{ndcg_at_k(recommendations, actual_items, count)}")

0.866947989864271


In [100]:
def novelty(recommendations, user_history, all_meditations):
    # Знаходимо нові медитації (ті, яких немає в історії користувача)
    new_items = set(recommendations) - set(user_history)
    # Повертаємо частку нових медитацій серед усіх рекомендацій
    return len(new_items) / len(recommendations)

In [101]:
print(f"{novelty(recommendations, actual_items, 10)}")

0.2


In [92]:
conn = pyodbc.connect("DRIVER={SQL Server};SERVER=DESKTOP-7S5VEFS\\SQLEXPRESS; DATABASE=NewMedit;Trusted_Connection=True;")
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT UserId FROM Ratings;")
results = cursor.fetchall()
userIds = [row[0] for row in results]
cursor.close()
conn.close()
print(userIds)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]


In [102]:
def personalization(all_recommendations):
    distances = []
    # Обчислюємо відстань Жаккара між списками рекомендацій для всіх пар користувачів
    for i in range(len(all_recommendations)):
        for j in range(i+1, len(all_recommendations)):
            # Порівнюємо рекомендації i-го користувача з рекомендаціями j-го користувача
            distances.append(jaccard(all_recommendations[i], all_recommendations[j]))
    # Повертаємо середню відстань між рекомендаціями різних користувачів
    return np.mean(distances)

In [107]:
user_recommendations = []
meditationProgramId = 6
count = 10
for userId in userIds:
    user_recommendations.append(get_hybrid_recomendation(userId, meditationProgramId, count))
print(user_recommendations)

Recommendations: {'recommendations': [3, 5, 4, 30, 8, 20, 23, 1, 19, 6]}
Recommendations: {'recommendations': [5, 30, 3, 4, 8, 23, 20, 1, 6, 9]}
Recommendations: {'recommendations': [8, 27, 3, 19, 6, 9, 15, 20, 14, 5]}
Recommendations: {'recommendations': [5, 3, 4, 8, 30, 20, 23, 1, 6, 19]}
Recommendations: {'recommendations': [5, 3, 4, 8, 30, 20, 23, 1, 9, 6]}
Recommendations: {'recommendations': [5, 3, 4, 8, 30, 20, 23, 1, 6, 19]}
Recommendations: {'recommendations': [5, 3, 4, 8, 30, 20, 23, 1, 6, 19]}
Recommendations: {'recommendations': [5, 3, 4, 8, 30, 20, 23, 1, 6, 13]}
Recommendations: {'recommendations': [5, 3, 4, 30, 8, 20, 23, 1, 27, 9]}
Recommendations: {'recommendations': [5, 3, 4, 8, 30, 20, 23, 1, 6, 9]}
Recommendations: {'recommendations': [3, 5, 4, 8, 30, 20, 23, 1, 6, 19]}
Recommendations: {'recommendations': [5, 3, 8, 4, 30, 20, 23, 1, 6, 16]}
Recommendations: {'recommendations': [5, 3, 4, 30, 8, 20, 23, 1, 7, 12]}
Recommendations: {'recommendations': [3, 5, 4, 30, 8,

In [108]:
print(f"{personalization(user_recommendations)}")

0.6357624831309042


In [41]:
update_models_url = 'http://localhost:5000/update-models'  # змініть URL на відповідний, якщо сервер працює на іншому порту або хості

# Надсилання POST-запиту без даних
response = requests.post(update_models_url)

# Обробка відповіді
if response.status_code == 200:
    print(response.text)
else:
    print('Error:', response.text)

Models updated sucessfull
